In [ ]:
import os
import torch
import sys
from google.colab import files

os.makedirs("models_export", exist_ok=True)
output_dir = "models_export"

print("🚀 1. Installing dependencies (2-3 min)...")
!pip install -q numpy==1.23.5 # Downgrade numpy for compatibility with older albumentations
!pip install -q torch torchvision onnx onnxruntime timm onnxscript
!pip install -q git+https://github.com/ChaoningZhang/MobileSAM.git
!pip install -q git+https://github.com/facebookresearch/segment-anything.git
!pip install -q onnxruntime-gpu

In [ ]:
import os
import torch
import torch.nn as nn
import torch.export # Required for torch.export.Dim

# Define output_dir here to ensure it's available
os.makedirs("models_export", exist_ok=True)
output_dir = "models_export"

print("\n📱 2. Downloading and Converting MobileSAM...")

# Ensure MobileSAM library is installed right before import
!pip install -q git+https://github.com/ChaoningZhang/MobileSAM.git
# Ensure onnxscript is available for torch.onnx.export
!pip install -q onnxscript

from mobile_sam import sam_model_registry, SamPredictor

if not os.path.exists("mobile_sam.pt"):
    !wget -q https://github.com/ChaoningZhang/MobileSAM/raw/master/weights/mobile_sam.pt

device = "cpu"
model_type = "vit_t"
sam_checkpoint = "mobile_sam.pt"

mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mobile_sam.to(device=device)
mobile_sam.eval()

print("   -> Exporting Encoder (image processing)...")

class EncoderWrapper(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder

    def forward(self, x):
        return self.encoder(x)

encoder_wrapper = EncoderWrapper(mobile_sam.image_encoder)
dummy_input = torch.randn(1, 3, 1024, 1024)

torch.onnx.export(
    encoder_wrapper,
    dummy_input,
    f"{output_dir}/mobilesam_encoder.onnx",
    input_names=['x'], # Changed 'input' to 'x' to match forward method argument
    output_names=['image_embeddings'],
    dynamic_shapes={
        'x': (1, 3, torch.export.Dim('height', min=8), torch.export.Dim('width', min=8))
    },
    opset_version=18, # Updated opset_version as per warning
    do_constant_folding=True
)

print("   -> Exporting Decoder (mask generation)....")

class DecoderWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.mask_decoder = model.mask_decoder
        self.prompt_encoder = model.prompt_encoder
        self.image_encoder = model.image_encoder

    def forward(self, image_embeddings, point_coords, point_labels):
        sparse_embeddings, dense_embeddings = self.prompt_encoder(
            points=(point_coords, point_labels),
            boxes=None,
            masks=None,
        )
        low_res_masks, iou_predictions = self.mask_decoder(
            image_embeddings=image_embeddings,
            image_pe=self.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False,
        )
        return low_res_masks, iou_predictions

decoder_wrapper = DecoderWrapper(mobile_sam)
embed_dim = mobile_sam.image_encoder.neck[0].out_channels
embed_size = 64

dummy_embeddings = torch.randn(1, embed_dim, embed_size, embed_size)
dummy_point_coords = torch.randn(1, 1, 2)
dummy_point_labels = torch.randint(0, 4, (1, 1))

torch.onnx.export(
    decoder_wrapper,
    (dummy_embeddings, dummy_point_coords, dummy_point_labels),
    f"{output_dir}/mobilesam_decoder.onnx",
    input_names=['image_embeddings', 'point_coords', 'point_labels'],
    output_names=['masks', 'iou_predictions'],
    dynamic_shapes={
        'image_embeddings': (1, embed_dim, embed_size, embed_size), # Explicitly set batch to 1
        'point_coords': (1, torch.export.Dim('num_points'), 2), # Explicitly set batch to 1, last dim is always 2
        'point_labels': (1, torch.export.Dim('num_points')) # Explicitly set batch to 1
    },
    opset_version=18, # Updated opset_version as per warning
    do_constant_folding=True
)

print("   -> Quantizing MobileSAM models...")
!pip install -q onnxruntime-gpu # Ensure onnxruntime is installed before import
from onnxruntime.quantization import quantize_dynamic, QuantType

# Quantize encoder
quantize_dynamic(
    model_input=f"{output_dir}/mobilesam_encoder.onnx",
    model_output=f"{output_dir}/mobilesam_encoder_quant.onnx",
    weight_type=QuantType.QUInt8
)

# Due to shape inference issues during quantization, keep decoder as FP32
# quantize_dynamic(
#     model_input=f"{output_dir}/mobilesam_decoder.onnx",
#     model_output=f"{output_dir}/mobilesam_decoder_quant.onnx",
#     weight_type=QuantType.QUInt8
# )

# os.remove(f"{output_dir}/mobilesam_encoder.onnx") # Keep FP32 version for now, might be useful
# os.remove(f"{output_dir}/mobilesam_decoder.onnx") # Keep FP32 version for now, might be useful

print("   -> Quantizing Encoder only (Decoder has shape conflicts)...")
print("   -> Simplifying Decoder (keeping FP32 for compatibility)...")

# Remove original FP32 encoder model as quantized version is available
os.remove(f"{output_dir}/mobilesam_encoder.onnx")

# Rename the FP32 decoder for clarity
os.rename(f"{output_dir}/mobilesam_decoder.onnx", f"{output_dir}/mobilesam_decoder_fp32.onnx")

print("   -> MobileSAM Ready!")
print("      Encoder: mobilesam_encoder_quant.onnx (quantized INT8)")
print("      Decoder: mobilesam_decoder_fp32.onnx (FP32 - better compatibility)")

In [ ]:
import sys
sys.path.insert(0, 'lama')

import yaml
import torch.nn as nn
from omegaconf import OmegaConf
from saicinpainting.training.trainers.default import DefaultInpaintingTrainingModule

print("   -> Loading PyTorch Model...")

with open('big-lama/config.yaml', 'r') as f:
    train_config = OmegaConf.create(yaml.safe_load(f))

checkpoint_path = 'big-lama/models/best.ckpt'
checkpoint = torch.load(checkpoint_path, map_location='cpu')

model = DefaultInpaintingTrainingTrainingModule(train_config)
model.load_state_dict(checkpoint['state_dict'], strict=False)
model.eval()

generator = model.generator
generator.eval()

class LaMaWrapper(nn.Module):
    def __init__(self, generator):
        super().__init__()
        self.generator = generator

    def forward(self, image, mask):
        batch = {'image': image, 'mask': mask}
        return self.generator(batch)

wrapper = LaMaWrapper(generator)
print("   -> Model loaded successfully!")

In [ ]:
import sys
sys.path.insert(0, 'lama')

import yaml
import torch.nn as nn
from omegaconf import OmegaConf
from saicinpainting.training.trainers.default import DefaultInpaintingTrainingModule

print("   -> Loading PyTorch Model...")

with open('big-lama/config.yaml', 'r') as f:
    train_config = OmegaConf.create(yaml.safe_load(f))

checkpoint_path = 'big-lama/models/best.ckpt'
checkpoint = torch.load(checkpoint_path, map_location='cpu')

model = DefaultInpaintingTrainingModule(train_config)
model.load_state_dict(checkpoint['state_dict'], strict=False)
model.eval()

generator = model.generator
generator.eval()

class LaMaWrapper(nn.Module):
    def __init__(self, generator):
        super().__init__()
        self.generator = generator

    def forward(self, image, mask):
        batch = {'image': image, 'mask': mask}
        return self.generator(batch)

wrapper = LaMaWrapper(generator)
print("   -> Model loaded successfully!")

In [ ]:
print("\n❤ 3. Baixando e Convertendo LaMa (Big-LaMa)...")

if not os.path.exists("lama"):
    !git clone -q https://github.com/advimman/lama.git

print("   -> Installing LaMa dependencies...")
# Uninstall any existing albumentations to ensure version 0.4.6 is used
!pip uninstall -y albumentations
!pip install -q omegaconf hydra-core albumentations==0.4.6 kornia webdataset scikit-image opencv-python easydict
!pip install -q wldhx.yadisk-direct pytorch-lightning
!pip install -q webdataset

if not os.path.exists("big-lama"):
    !curl -LJO https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip
    !unzip -q big-lama.zip -d big-lama

In [ ]:
sys.path.insert(0, 'lama')

import yaml
import torch.nn as nn
from omegaconf import OmegaConf
from saicinpainting.training.trainers.default import DefaultInpaintingTrainingModule

print("   -> Loading PyTorch Model...")

with open('big-lama/config.yaml', 'r') as f:
    train_config = OmegaConf.create(yaml.safe_load(f))

checkpoint_path = 'big-lama/models/best.ckpt'
checkpoint = torch.load(checkpoint_path, map_location='cpu')

model = DefaultInpaintingTrainingModule(train_config)
model.load_state_dict(checkpoint['state_dict'], strict=False)
model.eval()

generator = model.generator
generator.eval()

class LaMaWrapper(nn.Module):
    def __init__(self, generator):
        super().__init__()
        self.generator = generator

    def forward(self, image, mask):
        batch = {'image': image, 'mask': mask}
        return self.generator(batch)

wrapper = LaMaWrapper(generator)
print("   -> Model loaded successfully!")

In [ ]:
print("   -> Converting to ONNX (FP32)...")

dummy_image = torch.randn(1, 3, 512, 512)
dummy_mask = torch.randn(1, 1, 512, 512)
onnx_path = f"{output_dir}/lama.onnx"

with torch.no_grad():
    torch.onnx.export(
        wrapper,
        (dummy_image, dummy_mask),
        onnx_path,
        input_names=['image', 'mask'],
        output_names=['inpainted'],
        dynamic_axes={
            'image': {0: 'batch', 2: 'height', 3: 'width'},
            'mask': {0: 'batch', 2: 'height', 3: 'width'},
            'inpainted': {0: 'batch', 2: 'height', 3: 'width'}
        },
        opset_version=14,
        do_constant_folding=True
    )

print("   -> Quantizing LaMa (INT8)...")
from onnxruntime.quantization import quantize_dynamic, QuantType

quantize_dynamic(
    model_input=onnx_path,
    model_output=f"{output_dir}/lama_quant.onnx",
    weight_type=QuantType.QUInt8
)

os.remove(onnx_path)
print("   -> LaMa Ready!")

In [ ]:
print("\n📦 4. Compressing and Downloading...")
!ls -lh {output_dir}/
!zip -j models_postcard.zip {output_dir}/*
files.download('models_postcard.zip')
print("✅ Complete! Copy files from ZIP to 'project/frontend/public/models/'")

In [ ]:
# -*- coding: utf-8 -*-
"""LaMa_MobileSam.py

Standalone script to export MobileSAM and LaMa models to ONNX.
Compatible with Google Colab and local environments (Linux).
"""

import os
import sys
import subprocess
import shutil
import warnings

def install_dependencies():
    """Installs required dependencies if running in a notebook-like environment or requested."""
    print("🚀 Installing dependencies...")
    commands = [
        # Install numpy first, but we will enforce it again later
        "pip install -q \"numpy<2.0\"",
        "pip install -q torch torchvision onnx onnxruntime timm onnxscript",
        "pip install -q git+https://github.com/ChaoningZhang/MobileSAM.git",
        "pip install -q git+https://github.com/facebookresearch/segment-anything.git",
        "pip install -q omegaconf hydra-core albumentations==0.4.6 kornia webdataset scikit-image opencv-python easydict wldhx.yadisk-direct pytorch-lightning",
        # Force re-install albumentations to correct version
        "pip uninstall -y albumentations",
        "pip install -q albumentations==0.4.6",
        # CRITICAL: Force numpy < 2.0 at the end to ensure no other package upgraded it
        "pip install -q \"numpy<2.0\""
    ]
    for cmd in commands:
        print(f"Executing: {cmd}")
        subprocess.check_call(cmd, shell=True)

# Check args
if len(sys.argv) > 1 and sys.argv[1] == "--install":
    install_dependencies()

import torch
import torch.nn as nn

try:
    import mobile_sam
    from mobile_sam import sam_model_registry
except ImportError:
    print("❌ MobileSAM not found. Run with --install to install dependencies.")
    sys.exit(1)

# Setup Output Directory
os.makedirs("models_export", exist_ok=True)
output_dir = "models_export"

def download_file(url, filename):
    if not os.path.exists(filename):
        print(f"⬇️ Downloading {filename}...")
        subprocess.check_call(f"wget -q -O {filename} {url}", shell=True)
    else:
        print(f"✅ {filename} already exists.")

# ==========================================
# 1. MobileSAM Export
# ==========================================
print("\n📱 1. Processing MobileSAM...")

download_file("https://github.com/ChaoningZhang/MobileSAM/raw/master/weights/mobile_sam.pt", "mobile_sam.pt")

device = "cpu"
model_type = "vit_t"
checkpoint = "mobile_sam.pt"

print(f"   -> Loading {model_type}...")
mobile_sam_model = sam_model_registry[model_type](checkpoint=checkpoint)
mobile_sam_model.to(device=device)
mobile_sam_model.eval()

# --- Export Encoder ---
print("   -> Exporting Encoder...")

class EncoderWrapper(nn.Module):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder

    def forward(self, x):
        return self.encoder(x)

encoder_wrapper = EncoderWrapper(mobile_sam_model.image_encoder)
dummy_input = torch.randn(1, 3, 1024, 1024)

encoder_onnx_path = os.path.join(output_dir, "mobilesam_encoder.onnx")

# NOTE: Static shapes for Encoder (1x3x1024x1024)
with torch.no_grad():
    torch.onnx.export(
        encoder_wrapper,
        dummy_input,
        encoder_onnx_path,
        input_names=['x'],
        output_names=['image_embeddings'],
        opset_version=18, # Updated to 18 to match PyTorch defaults and avoid conversion errors
        do_constant_folding=True
    )

# --- Export Decoder ---
print("   -> Exporting Decoder...")

class DecoderWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.mask_decoder = model.mask_decoder
        self.prompt_encoder = model.prompt_encoder

    def forward(self, image_embeddings, point_coords, point_labels):
        sparse_embeddings, dense_embeddings = self.prompt_encoder(
            points=(point_coords, point_labels),
            boxes=None,
            masks=None,
        )
        low_res_masks, iou_predictions = self.mask_decoder(
            image_embeddings=image_embeddings,
            image_pe=self.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False,
        )
        return low_res_masks, iou_predictions

decoder_wrapper = DecoderWrapper(mobile_sam_model)

embed_dim = mobile_sam_model.image_encoder.neck[0].out_channels
embed_size = 64

dummy_embeddings = torch.randn(1, embed_dim, embed_size, embed_size)
dummy_point_coords = torch.randn(1, 2, 2) # Fixed 2 points
dummy_point_labels = torch.randn(1, 2)

decoder_onnx_path = os.path.join(output_dir, "mobilesam_decoder.onnx")

# NOTE: Static shapes for Decoder (1 image, 2 points)
with torch.no_grad():
    torch.onnx.export(
        decoder_wrapper,
        (dummy_embeddings, dummy_point_coords, dummy_point_labels),
        decoder_onnx_path,
        input_names=['image_embeddings', 'point_coords', 'point_labels'],
        output_names=['masks', 'iou_predictions'],
        opset_version=18, # Updated to 18
        do_constant_folding=True
    )

# --- Quantization MobileSAM ---
print("   -> Quantizing MobileSAM Encoder...")
from onnxruntime.quantization import quantize_dynamic, QuantType

encoder_quant_path = os.path.join(output_dir, "mobilesam_encoder_quant.onnx")
quantize_dynamic(
    model_input=encoder_onnx_path,
    model_output=encoder_quant_path,
    weight_type=QuantType.QUInt8
)

# Clean up FP32 encoder
if os.path.exists(encoder_onnx_path):
    os.remove(encoder_onnx_path)

# Rename Decoder (keep FP32)
decoder_fp32_path = os.path.join(output_dir, "mobilesam_decoder_fp32.onnx")
if os.path.exists(decoder_onnx_path):
    # Rename/Move
    if os.path.exists(decoder_fp32_path):
        os.remove(decoder_fp32_path)
    shutil.move(decoder_onnx_path, decoder_fp32_path)

print("✅ MobileSAM Export Complete.")


# ==========================================
# 2. LaMa Export
# ==========================================
print("\n🎨 2. Processing LaMa...")

# CHECK NUMPY VERSION BEFORE IMPORTING LAMA
import numpy as np
print(f"   -> Current NumPy version: {np.__version__}")
if int(np.__version__.split('.')[0]) >= 2:
    print("⚠️ NumPy 2.x detected! This breaks LaMa.")
    print("   -> Downgrading NumPy to <2.0...")
    subprocess.check_call("pip install -q \"numpy<2.0\"", shell=True)
    # We need to reload numpy, but in a script that's hard.
    # However, since we haven't imported LaMa modules yet,
    # the new process spawned by imports might pick it up, or we might need a restart.
    # But usually, 'import' caches the module.
    # Let's hope the environment allows hot-swapping or user ran with --install.
    print("   ❗ IF THIS FAILS, PLEASE RESTART RUNTIME AND RUN AGAIN ❗")

if not os.path.exists("lama"):
    print("   -> Cloning LaMa...")
    subprocess.check_call("git clone -q https://github.com/advimman/lama.git", shell=True)

if not os.path.exists("big-lama"):
    print("   -> Downloading Big-LaMa weights...")
    subprocess.check_call("curl -LJO https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip", shell=True)
    subprocess.check_call("unzip -q -o big-lama.zip -d big-lama", shell=True)

# Add lama to path
sys.path.insert(0, os.path.abspath('lama'))

try:
    import yaml
    from omegaconf import OmegaConf
    from saicinpainting.training.trainers.default import DefaultInpaintingTrainingModule
except ImportError as e:
    print(f"❌ Failed to import LaMa modules: {e}")
    print("   This is usually due to NumPy 2.0 incompatibility.")
    print("   Try running: pip install \"numpy<2.0\" and RESTART THE RUNTIME.")
    sys.exit(1)
except AttributeError as e:
    print(f"❌ Numpy Version Error: {e}")
    print("   You must downgrade numpy: pip install \"numpy<2.0\"")
    sys.exit(1)

print("   -> Loading LaMa Model...")
# Handle nested folder structure if it occurs
config_path = 'big-lama/config.yaml'
checkpoint_path = 'big-lama/models/best.ckpt'

if not os.path.exists(config_path):
     if os.path.exists('big-lama/big-lama/config.yaml'):
        config_path = 'big-lama/big-lama/config.yaml'
        checkpoint_path = 'big-lama/big-lama/models/best.ckpt'

with open(config_path, 'r') as f:
    train_config = OmegaConf.create(yaml.safe_load(f))

# Disable strict loading
train_config.training_model.predict_only = True
train_config.visualizer.kind = 'noop'

checkpoint = torch.load(checkpoint_path, map_location='cpu')

lama_model = DefaultInpaintingTrainingModule(train_config)
lama_model.load_state_dict(checkpoint['state_dict'], strict=False)
lama_model.eval()

class LaMaWrapper(nn.Module):
    def __init__(self, generator):
        super().__init__()
        self.generator = generator

    def forward(self, image, mask):
        # LaMa generator expects a dict
        batch = {'image': image, 'mask': mask}
        return self.generator(batch)

lama_wrapper = LaMaWrapper(lama_model.generator)

dummy_image = torch.randn(1, 3, 512, 512)
dummy_mask = torch.randn(1, 1, 512, 512)

lama_onnx_path = os.path.join(output_dir, "lama.onnx")

print("   -> Exporting LaMa to ONNX (FP32)...")
# User requested stability/quality over size (200MB is fine).
# We will NOT quantize LaMa to avoid quality loss and potential INT8 operator issues.
with torch.no_grad():
    torch.onnx.export(
        lama_wrapper,
        (dummy_image, dummy_mask),
        lama_onnx_path,
        input_names=['image', 'mask'],
        output_names=['inpainted'],
        opset_version=18,
        do_constant_folding=True
    )

# Skipping Quantization for LaMa as requested
# lama_quant_path = os.path.join(output_dir, "lama_quant.onnx")
# quantize_dynamic(...)

print("✅ LaMa Export Complete (FP32 - High Quality).")

# Compress
print("\n📦 Compressing models...")
zip_filename = "models_postcard.zip"
shutil.make_archive("models_postcard", 'zip', output_dir)
print(f"🎉 Done! Download {zip_filename}.zip and unzip to project/frontend/public/models/")


/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.12/dist-packages/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.12/dist-packages/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m


📱 1. Processing MobileSAM...
✅ mobile_sam.pt already exists.
   -> Loading vit_t...
   -> Exporting Encoder...


W0130 13:19:34.624000 20039 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0130 13:19:34.626000 20039 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0130 13:19:34.630000 20039 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0). Treating as an Input.
W0130 13:19:34.632000 20039 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0). Treating as an Input.


[torch.onnx] Obtain model graph for `EncoderWrapper([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `EncoderWrapper([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 66 of general pattern rewrite rules.
   -> Exporting Decoder...


W0130 13:19:45.179000 20039 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0130 13:19:45.181000 20039 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0130 13:19:45.185000 20039 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0). Treating as an Input.
W0130 13:19:45.190000 20039 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0). Treating as an Input.


[torch.onnx] Obtain model graph for `DecoderWrapper([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `DecoderWrapper([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 9 of general pattern rewrite rules.


   -> Quantizing MobileSAM Encoder...
✅ MobileSAM Export Complete.

🎨 2. Processing LaMa...
   -> Current NumPy version: 1.26.4
Detectron v2 is not installed
   -> Loading LaMa Model...


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint was not an allowed global by default. Please use `torch.serialization.add_safe_globals([pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint])` or the `torch.serialization.safe_globals([pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint])` context manager to allowlist t